# Example Duckweed Pickup Workflow

This notebook offers an example workflow to pick up & drop of duckweed fronds using a Jubilee outfitted with USB microscope and 10cc syring tools.


In [ ]:
from MachineUtils import *
from CameraUtils import *
import os
import yaml

In [ ]:
# We'll need to open the USB microscope as a camera by its index
# Check your valid camera indices with this command
# if you have multiple, you'll have to try them until you pick the right one

getCameraIndices();

In [ ]:
# Change this value to the camera index from above
# Check that it gets a frame from the correct camera
# %matplotlib inline
camIdx = 0
frame = getFrame(camIdx); 
showFrame(frame)

In [ ]:
# Read in your calibration file

with open("../calibration/distance-calibration.yml", "r") as stream:
    try:
        distanceCal = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
# grab the x & y values
# if you've calibrated for distances other than z=50mm, change this accordingly

rx = distanceCal['z_50']['x']
ry = distanceCal['z_50']['y']

In [ ]:
# Setup your machine connection
# List available ports in thie cell

ports = serial.tools.list_ports.comports()
print([port.name for port in ports]) 

In [ ]:
# Choose the correct port from above and establish connection with machine
port = '/dev/ttyACM0'
m = MachineCommunication(port)

In [ ]:
# image (a section of) the bed, stepping by the microscope viewport size
# To make sure images are stitched together in the correct order, increment in the +x/+y direction

xStart = 112 # edit these to image the correct portion of your bed
xEnd = 135
yStart = 115
yEnd = 185

images = []
row = 0
m.toolChange(1) # assuming microscope is T1
for x in np.arange(xStart, xEnd, rx):
    images.insert(row, [])
    for y in np.arange(yStart, yEnd, ry):
        m.moveTo(x=x, y=y)
        f = getFrame(camIdx)
        images[row].append(f)
    row+=1

In [ ]:
# stitch the images together
stitched = []
for row in range(len(images)):
    stitched.insert(0, cv2.vconcat(images[row])) # insert at beginning to ensure order is correct
full = cv2.hconcat(stitched)
cv2.imwrite('/home/pi/duckweed/test1.jpg', full)
pixelHeight, pixelWidth, z = full.shape # pixel dimensions of full img. height/width are flipped due to orientation of microscope

In [ ]:
showFrame(cv2.imread('/home/pi/duckweed/test1.jpg'))
pixelWidth

In [ ]:
clickX, clickY = selectPoint(cv2.imread('/home/pi/duckweed/test1.jpg'))

In [ ]:
# move to the selected point
numY = len(images[0]) # number of images taken in Y bed direction
numX = len(stitched) # number of images taken in X bed direction

# calculate the real world bounds of the stitched image
# the minimum values are the starting values, minus half the microscopes viewport in that direction
minX = xStart - rx/2
minY = yStart - ry / 2
# max values are the number of imgs taken in each direction offset by start values
maxX = minX + numX * rx
maxY = minY + numY * ry

dx = np.interp(clickX, [0, pixelWidth], [maxX, minX]) # x pixel dimension is flipped relative to real world
dy = np.interp(clickY, [0, pixelHeight], [minY, maxY])
print(dx)
print(dy)


In [ ]:
m.moveTo(x=dx, y=dy)

In [ ]:
check = getFrame()
showFrame(check, grid=True)

In [ ]:
m.toolChange(2)
offset = (-3.7, 5.75)
m.moveTo(x=dx+offset[0], y=dy+offset[1])

In [ ]:
# dip down to the duckweed
# find manually & change based on your setup
m.moveTo(z=-40.9)